In [ ]:
import numpy as np
from scipy import stats
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from matplotlib.transforms import Bbox
import matplotlib
import plotly.graph_objects as go
import pickle as pkl
import keras
import tensorflow_addons as tfa
import os
from GraphFuncsV4 import*
from matplotlib import cm
from tensorflow.keras.layers import *

In [ ]:
#Import model
model_id = 'GatedCNN_1.0e-03_8_1_2'
modelname = 'realdata_%s' % model_id
model = keras.models.load_model('./results/%s.h5' % model_id)

#Create folders for plots
try:
    os.mkdir("./Plots/%s" % modelname)
    os.mkdir("./Plots/%s/Model" % modelname)
    os.mkdir("./Plots/%s/Model/Single Model" % modelname)
    os.mkdir("./Plots/%s/Model(3)" % modelname)
    os.mkdir("./Plots/%s/Model(3)/Single Model" % modelname)
    os.mkdir("./Plots/%s/Model(5)" % modelname)
    os.mkdir("./Plots/%s/Model(5)/Single Model" % modelname)
    os.mkdir("./Plots/%s/Comparison" % modelname)
    os.mkdir("./Plots/%s/Comparison/Single Model" % modelname)
    os.mkdir("./Plots/%s/Chi-Squared" % modelname)
    os.mkdir("./Plots/%s/Chi-Squared/Single Model" % modelname)
except OSError:
    print("Folder %s already exists" % modelname)

In [ ]:
#Import scaler and real images and paramters
savedir = "./results/"
yscaler = pkl.load(open(savedir + "yscaler", "rb"))
testin = pkl.load(open(savedir + "realdatain", "rb"))
testout = pkl.load(open(savedir + "realdataout", "rb"))
testout = testout.numpy()

In [ ]:
true = testout[:,5] #Get only defocus parameter
true *= 2 #Scale by 2 for conversion from defocus to micrometers (approximately a factor of 2)
pred = model.predict(testin) #Make model predictions on real data
pred = yscaler.inverse_transform(pred)[:,0] #Unscale the predicted parameters
pred *= 2 #Scale by 2 for conversion from defocus to micrometers (approximately a factor of 2)
chisquare_pred = pkl.load(open("./results_single/single_parameters.pkl", "rb")) #Get chi-square predictions of model
chisquare_pred = np.array(chisquare_pred)[:,5] #Get only defocus
chisquare_pred *= 2 #Scale by 2 for conversion from defocus to micrometers (approximately a factor of 2)

In [ ]:
#Check that number of images is same
print(np.shape(pred))
print(np.shape(true))
print(np.shape(chisquare_pred))

In [ ]:
#Get names for variables of interest
varname = [
#            "Amplitude", 
#            "Decay Coefficient", 
#            "Spherical Aberration", 
#            "Astigmatism", 
#            "Principal Axis Angle", 
           "Defocus", 
#            "Detuning"
          ]

In [ ]:
bins = 10 #Number of plot bins
space = 'Single Model' #Name of plot image sub-folder

#Generate plots for model from single-OD images
for i in range(len(varname)):
    lower, upper = getlowerupper(pred[:], true[:])
    
    preds = np.array([pred[:], chisquare_pred[:]])
    
    prednames = ['ML Model', 'Chi-Squared']
    
    histo(modelname, 
          'Comparison', 
          space, 
          true[:], 
          preds, 
          varname[i], 
          prednames, 
          lower, 
          upper, 
          bins)
    
    heatmap(modelname, 
            'Model', 
            space, 
            true[:], 
            pred[:], 
            varname[i], 
            lower, 
            upper, 
            bins)
    
    heatmap(modelname, 
            'Chi-Squared', 
            space, 
            true[:],
            chisquare_pred[:], 
            varname[i], 
            lower, 
            upper, 
            bins)
    
    resmeans, stddevs, bincenters, samples, resmean, std = \
    plotgaussian(modelname, 
                 'Model', 
                 space, 
                 true[:], 
                 pred[:], 
                 varname[i], 
                 lower, 
                 upper, 
                 bins)
            
    chisquare_resmeans, chisquare_stddevs, chisquare_bincenters, chisquare_samples, chisquare_resmean, chisquare_std = \
    plotgaussian(modelname, 
                 'Chi-Squared', 
                 space, 
                 true[:], 
                 chisquare_pred[:], 
                 varname[i], 
                 lower, 
                 upper, 
                 bins)
    
    scatter(modelname,
            space,
            [resmeans, chisquare_resmeans], 
            [bincenters, chisquare_bincenters], 
            [samples, chisquare_samples], 
            varname[i], 
            ['ML Model', 'Chi-Squared'], 
            'Mean')
    
    
    scatter(modelname,
            space,
            [stddevs, chisquare_stddevs], 
            [bincenters, chisquare_bincenters], 
            [samples, chisquare_samples], 
            varname[i], 
            ['ML Model', 'Chi-Squared'],
            'Stddev')
    
    
    confinttable(modelname, 
                 space, 
                 'Model', 
                 resmean, 
                 std, 
                 varname[i], 
                 sum(samples), 
                 lower, 
                 upper)
    
    confinttable(modelname, 
                 space, 
                 'Chi-Squared', 
                 chisquare_resmean, 
                 chisquare_std, 
                 varname[i], 
                 sum(chisquare_samples), 
                 lower, 
                 upper)

In [ ]:
#Import 3-OD images averaged data sets
threeimages = pkl.load(open(savedir + "3images", "rb"))
threeimparams = pkl.load(open(savedir + "3imparams", "rb"))
threeimparams = threeimparams.numpy()
true = threeimparams.copy()
true *= 2
pred3 = model.predict(threeimages)
pred3 = yscaler.inverse_transform(pred3)[:,0]
pred3 *= 2

In [ ]:
print(np.shape(true))

In [ ]:
#Generate plots for 3-images averaged data sets
bins = 10
space = 'Single Model'

for i in range(len(varname)):
    lower, upper = getlowerupper(pred3[:], true[:])
    
    prednames = ['ML Model(3)', 'Chi-Squared']
#     prednames = ['ML Model']
    
    histo(modelname, 
          'Comparison', 
          space, 
          true[:], 
          preds, 
          varname[i], 
          prednames, 
          lower, 
          upper, 
          bins)
    
    heatmap(modelname, 
            'Model(3)', 
            space, 
            true[:], 
            pred3[:], 
            varname[i], 
            lower, 
            upper, 
            bins)   
    
    resmeans, stddevs, bincenters, samples, resmean, std = \
    plotgaussian(modelname, 
                 'Model(3)', 
                 space, 
                 true[:], 
                 pred3[:], 
                 varname[i], 
                 lower, 
                 upper, 
                 bins)    
    
    confinttable(modelname, 
                 space, 
                 'Model(3)', 
                 resmean, 
                 std, 
                 varname[i], 
                 sum(samples), 
                 lower, 
                 upper)

In [ ]:
#Import 5-OD images averaged data sets
fiveimages = pkl.load(open(savedir + "5images", "rb"))
fiveimparams = pkl.load(open(savedir + "5imparams", "rb"))
fiveimparams = fiveimparams.numpy()
true = fiveimparams.copy()
true *= 2
pred5 = model.predict(fiveimages)
pred5 = yscaler.inverse_transform(pred5)[:,0]
pred5 *= 2

In [ ]:
#Generate plots for 5 images averaged data sets
bins = 10
space = 'Single Model'

for i in range(len(varname)):
    lower, upper = getlowerupper(pred5[:], true[:])
    
    prednames = ['ML Model(5)', 'Chi-Squared']
    
    histo(modelname, 
          'Comparison', 
          space, 
          true[:], 
          preds, 
          varname[i], 
          prednames, 
          lower, 
          upper, 
          bins)
    
    heatmap(modelname, 
            'Model(5)', 
            space, 
            true[:], 
            pred5[:], 
            varname[i], 
            lower, 
            upper, 
            bins)   
    
    resmeans, stddevs, bincenters, samples, resmean, std = \
    plotgaussian(modelname, 
                 'Model(5)', 
                 space, 
                 true[:], 
                 pred5[:], 
                 varname[i], 
                 lower, 
                 upper, 
                 bins)    
    
    confinttable(modelname, 
                 space, 
                 'Model(5)', 
                 resmean, 
                 std, 
                 varname[i], 
                 sum(samples), 
                 lower, 
                 upper)

In [ ]:
imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

K_x, K_y = pkl.load(open(savedir + "K_xK_y", "rb"))

from NPSmethods2 import pupilFunc, getFreq

#Generate M2k_Fit
def make_M2k_Fit(paras, imgSysData):
    
    A, tau, S0, alpha, phi, beta, delta_s = paras
    _, _, K_X, K_Y = getFreq(imgSysData["CCDPixelSize"],
                                      imgSysData["magnification"], (100,100))
    d = imgSysData["wavelen"] / (2*np.pi*imgSysData["NA"])
    R_p, Theta_p = np.abs(K_X + 1j*K_Y) * d, np.angle(K_X + 1j*K_Y)
    p1 = pupilFunc(R_p, Theta_p + np.pi, tau, S0, alpha, phi, beta)
    p2 = np.conj(pupilFunc(R_p, Theta_p, tau, S0, alpha, phi, beta)) * \
            np.exp(-2*1j*delta_s)
    PSF = (p1 + p2) / (2 * np.cos(delta_s))
    M2k = np.abs(PSF)**2
    M2k_Fit = A * M2k
    M2k_Fit[M2k_Fit.shape[0]//2, M2k_Fit.shape[1]//2] = 0
    
    return M2k_Fit

try:
    os.mkdir("./results/Predicted Images ML/")
except OSError:
    print("Folder Predicted Images ML already exists")
    
try:
    os.mkdir("./results/Predicted Images Chi-Squared/")
except OSError:
    print("Folder Predicted Images Chi-Squared already exists")

In [ ]:
#Generate predicted images based on supplied defocus predictions from model and chi-square fit method
for paranum in range(len(testout)):
    testout[paranum][5]  = pred[paranum]
    M2k_Fit = make_M2k_Fit(testout[paranum], imgSysData)
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, M2k_Fit, cmap=cm.jet, vmin=0, vmax=M2k_Fit.max())
    plt.colorbar(im)
    fig.savefig(savedir + 'Predicted Images ML/Single' + str(paranum + 1))
    plt.close(fig)
    
for paranum in range(len(testout)):
    testout[paranum][5]  = chisquare_pred[paranum]
    M2k_Fit = make_M2k_Fit(testout[paranum], imgSysData)
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, M2k_Fit, cmap=cm.jet, vmin=0, vmax=M2k_Fit.max())
    plt.colorbar(im)
    fig.savefig(savedir + 'Predicted Images Chi-Squared/Single' + str(paranum + 1))
    plt.close(fig)